# Feature Engineering - Bike Share System in the SF Bay Area

Author: Owen Hsu

## Table of content

1. Feature Overview
2. Loading and Setup
3. Assessment
4. Categorical Variables
5. Scaling and Train/Test Split

## Feature Overview



 Columns                    | Description 
|:---------:                |:-----------:                                                              |
|  Time                     |  Recorded Date and Time                                                   |
|  Station ID               |  Station ID                                                               |
|  Bikes Available          |  The number of bikes available at the station                             | 
|  Dock Number              |  The number of docks at the station                                       | 
|  Mean Dew Point           |  The average dew point of the date                                        | 
|  Mean Humidity            |  The average humidity of the date                                         | 
|  Mean Sea Level Pressure  |  The average sea level pressure of the date                               | 
|  Mean Visibility          |  The average visibility of the date                                       | 
|  Mean Wind Speed          |  The average wind speed of the date                                       | 
|  precipitation_inches     |  Determine whether the date of each record has precipitation              | 
|  Cloud Cover              |  The fraction of the sky covered by clouds on given date                  | 
|  Zip Code                 |  The zip code for the weather record                                      |
|  Usage Rate Category      |  Usage Rate Category of the station                                       |
|  Holiday                  |  Determine whether the date of each record is a holiday or not            |
|  Is Weekend or Holiday    |  Determine whether the date of each record is a holiday/weekend or not    |
|  Hour                     |  Recorded Time (Hour)                                                     |


## Loading and Setup

In [28]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler 
# Filter warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [101]:
# Load the dataset
df = pd.read_parquet('data/BikeData_after_EDA.parquet')

## Assessment

In [102]:
# Set display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [103]:
# Print the shape of the data
df.shape

(7337194, 16)

In [104]:
# Look at the first 5 rows
df.head()

,time,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,usage_rate_category,holiday,is_weekend_or_holiday,hour
0,2014-05-01 00:00:00,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,Median,0,0,0
1,2014-05-01 00:05:00,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,Median,0,0,0
2,2014-05-01 00:10:00,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,Median,0,0,0
3,2014-05-01 00:15:00,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,Median,0,0,0
4,2014-05-01 00:20:00,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,Median,0,0,0


In [105]:
# Get a quick overview of dataset variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7337194 entries, 0 to 7337193
Data columns (total 16 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   time                            datetime64[ns]
 1   station_id                      int64         
 2   bikes_available                 int64         
 3   dock_count                      int64         
 4   mean_dew_point_f                float64       
 5   mean_humidity                   float64       
 6   mean_sea_level_pressure_inches  float64       
 7   mean_visibility_miles           float64       
 8   mean_wind_speed_mph             float64       
 9   precipitation_inches            int64         
 10  cloud_cover                     float64       
 11  zip_code                        int64         
 12  usage_rate_category             object        
 13  holiday                         int64         
 14  is_weekend_or_holiday           int64         
 15

In [106]:
# Sanity check to make sure we don't have any NAs
df.isna().sum()

time                              0
station_id                        0
bikes_available                   0
dock_count                        0
mean_dew_point_f                  0
mean_humidity                     0
mean_sea_level_pressure_inches    0
mean_visibility_miles             0
mean_wind_speed_mph               0
precipitation_inches              0
cloud_cover                       0
zip_code                          0
usage_rate_category               0
holiday                           0
is_weekend_or_holiday             0
hour                              0
dtype: int64

In [107]:
# Get a statistical summary of the dataset
df.describe()

,time,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,holiday,is_weekend_or_holiday,hour
count,7337194,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06,7.337194e+06
mean,2014-10-30 13:33:29.071412224,4.299356e+01,8.296316e+00,1.765689e+01,4.968303e+01,6.859634e+01,3.001985e+01,9.598144e+00,6.647163e+00,1.599805e-01,3.262926e+00,9.433987e+04,2.747494e-02,3.128430e-01,1.150000e+01
min,2014-05-01 00:00:00,2.000000e+00,0.000000e+00,1.100000e+01,1.300000e+01,2.400000e+01,2.963000e+01,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.404100e+04,0.000000e+00,0.000000e+00,0.000000e+00
25%,2014-07-31 01:55:00,2.400000e+01,5.000000e+00,1.500000e+01,4.600000e+01,6.300000e+01,2.993000e+01,1.000000e+01,4.000000e+00,0.000000e+00,1.000000e+00,9.410700e+04,0.000000e+00,0.000000e+00,6.000000e+00
50%,2014-10-30 20:45:00,4.200000e+01,8.000000e+00,1.500000e+01,5.000000e+01,6.900000e+01,3.000000e+01,1.000000e+01,6.000000e+00,0.000000e+00,3.000000e+00,9.410700e+04,0.000000e+00,0.000000e+00,1.100000e+01
75%,2015-01-29 20:30:00,6.400000e+01,1.100000e+01,1.900000e+01,5.500000e+01,7.500000e+01,3.011000e+01,1.000000e+01,9.000000e+00,0.000000e+00,5.000000e+00,9.430100e+04,0.000000e+00,1.000000e+00,1.700000e+01
max,2015-04-30 23:55:00,8.400000e+01,2.700000e+01,2.700000e+01,6.500000e+01,9.600000e+01,3.041000e+01,2.000000e+01,2.300000e+01,1.000000e+00,8.000000e+00,9.511300e+04,1.000000e+00,1.000000e+00,2.300000e+01
std,NaN,2.399402e+01,4.173161e+00,3.982221e+00,6.848807e+00,1.073235e+01,1.291384e-01,1.313508e+00,3.303102e+00,3.665880e-01,2.229327e+00,4.248337e+02,1.634628e-01,4.636511e-01,6.920271e+00


#### Numerical Features

In [108]:
# Identify the numerical columns
num_cols = df.select_dtypes(include=['number']).columns.tolist()

# Show the list of numerical columns
num_cols

['station_id',
 'bikes_available',
 'dock_count',
 'mean_dew_point_f',
 'mean_humidity',
 'mean_sea_level_pressure_inches',
 'mean_visibility_miles',
 'mean_wind_speed_mph',
 'precipitation_inches',
 'cloud_cover',
 'zip_code',
 'holiday',
 'is_weekend_or_holiday',
 'hour']

In [109]:
# Count the number of numerical columns
print(f'Number of numerical columns: {len(num_cols)}')

Number of numerical columns: 14


#### Categorical Features

In [110]:
# Identify the categorical columns
cate_cols = df.select_dtypes(include='object').columns.tolist()

# Show the list of categorical columns
cate_cols

['usage_rate_category']

In [111]:
# Count the number of categorical columns
print(f'Number of numerical columns: {len(cate_cols)}')

Number of numerical columns: 1


#### Datetime Features

In [112]:
# Identify the datetime columns
dat_cols = df.select_dtypes(include=['datetime']).columns.tolist()

# Show the list of numerical columns
dat_cols

['time']

In [113]:
# Count the number of datetime columns
print(f'Number of datetime columns: {len(dat_cols)}')

Number of datetime columns: 1


## Categorical Variables

#### Mapping the target variable to numerical values

We only have one categorical variable in our dataset. Let's proceed with manually encoding this variable.

In [114]:
# Get the unique values in the 'usage_rate_category' column
df['usage_rate_category'].unique()

array(['Median', 'High', 'Low'], dtype=object)

We will convert column 'usage_rate_category' into a numerical one, where 1 represents low usage, 2 represents median usage, and 3 represents high usage.

In [115]:
# Convert the 'usage_rate_category' column to a numerical one
df['usage_rate_category'] = df['usage_rate_category'].replace({'Low': 1, 'Median': 2, 'High': 3}).astype(int)

# Verify the conversion
df.sample(10)

,time,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,usage_rate_category,holiday,is_weekend_or_holiday,hour
2409346,2015-04-29 03:30:00,83,6,15,47.0,69.0,30.00,10.0,4.0,0,1.0,94063,2,0,0,3
5088548,2014-09-19 02:25:00,70,17,19,60.0,79.0,29.90,10.0,7.0,0,5.0,94107,1,0,0,2
5243084,2014-10-05 16:25:00,42,8,15,52.0,55.0,29.92,10.0,5.0,0,1.0,94107,2,0,1,16
870877,2014-11-06 08:35:00,14,9,19,50.0,62.0,30.19,10.0,3.0,0,3.0,95113,2,0,0,8
1532761,2015-03-30 13:05:00,9,6,15,48.0,70.0,30.05,10.0,6.0,0,3.0,95113,2,0,0,13
4818838,2014-08-23 18:00:00,49,7,19,55.0,64.0,29.91,10.0,9.0,0,3.0,94107,2,0,1,18
5118443,2014-09-22 21:40:00,68,5,19,57.0,66.0,30.00,10.0,9.0,0,4.0,94107,3,0,0,21
6645002,2015-02-21 10:55:00,47,1,19,47.0,67.0,29.91,10.0,5.0,0,2.0,94107,3,0,1,10
2592294,2014-07-29 09:10:00,31,8,15,59.0,69.0,30.04,10.0,7.0,0,2.0,94041,2,0,0,9
2911844,2015-01-04 07:15:00,33,7,15,36.0,70.0,30.39,8.0,2.0,1,0.0,94041,2,0,1,7


In [116]:
# Verify if there is any remaining categorical columns in the dataset
df.select_dtypes(['object']).columns

Index([], dtype='object')

## Datetime Variables

#### Extracting datetime variables

We have one datetime variable. Let's extract useful date/time components such as year, month, day, and minute. Since we already have the 'hour' column, we will drop it first to ensure the date/time information is organized in the following order in our dataframe: year, month, day, hour, minute.

In [117]:
# Drop the column 'hour'
df = df.drop('hour', axis=1)

# Extract datetime components and create new columns
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute

# Verify the conversion
df.sample(15)

,time,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,usage_rate_category,holiday,is_weekend_or_holiday,year,month,day,hour,minute
1794124,2014-06-27 14:45:00,26,8,15,55.0,77.0,29.98,10.0,6.0,0,1.0,94063,2,0,0,2014,6,27,14,45
2145942,2014-12-20 13:50:00,21,6,15,55.0,93.0,30.19,8.0,1.0,0,8.0,94063,2,0,1,2014,12,20,13,50
1930362,2014-09-03 15:55:00,23,7,15,56.0,74.0,29.76,10.0,4.0,0,5.0,94063,2,0,0,2014,9,3,15,55
7276524,2015-04-24 02:25:00,77,7,27,47.0,69.0,29.99,10.0,19.0,1,5.0,94107,3,0,0,2015,4,24,2,25
3750079,2014-05-08 21:15:00,82,2,15,50.0,73.0,30.06,10.0,10.0,1,7.0,94107,3,0,0,2014,5,8,21,15
6113487,2014-12-30 22:00:00,58,9,19,23.0,53.0,30.16,10.0,17.0,0,0.0,94107,2,0,0,2014,12,30,22,0
6955274,2015-03-23 05:55:00,82,3,15,49.0,71.0,30.25,10.0,10.0,1,6.0,94107,3,0,0,2015,3,23,5,55
1894258,2014-08-16 07:15:00,24,10,15,57.0,78.0,30.01,10.0,4.0,0,4.0,94063,2,0,1,2014,8,16,7,15
6173292,2015-01-05 13:45:00,56,12,19,38.0,68.0,30.32,9.0,3.0,0,4.0,94107,2,0,0,2015,1,5,13,45
4227846,2014-06-25 06:25:00,61,14,27,55.0,79.0,29.99,10.0,13.0,1,7.0,94107,2,0,0,2014,6,25,6,25


In [118]:
# Drop the column 'time'
df = df.drop('time', axis=1)

In [119]:
# Look at the first 5 rows
df.head()

,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,usage_rate_category,holiday,is_weekend_or_holiday,year,month,day,hour,minute
0,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,2,0,0,2014,5,1,0,0
1,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,2,0,0,2014,5,1,0,5
2,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,2,0,0,2014,5,1,0,10
3,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,2,0,0,2014,5,1,0,15
4,2,12,27,45.0,41.0,30.06,10.0,6.0,0,3.0,95113,2,0,0,2014,5,1,0,20


In [120]:
# Verify if there is any remaining datetime columns in the dataset
df.select_dtypes(['datetime']).columns

Index([], dtype='object')

In [121]:
# Double check the shape of the dataset before train/test split
df.shape

(7337194, 19)

In [122]:
# Get a quick overview of dataset variables
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7337194 entries, 0 to 7337193
Data columns (total 19 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   station_id                      int64  
 1   bikes_available                 int64  
 2   dock_count                      int64  
 3   mean_dew_point_f                float64
 4   mean_humidity                   float64
 5   mean_sea_level_pressure_inches  float64
 6   mean_visibility_miles           float64
 7   mean_wind_speed_mph             float64
 8   precipitation_inches            int64  
 9   cloud_cover                     float64
 10  zip_code                        int64  
 11  usage_rate_category             int64  
 12  holiday                         int64  
 13  is_weekend_or_holiday           int64  
 14  year                            int32  
 15  month                           int32  
 16  day                             int32  
 17  hour                       

## Train/Test Split and Scaling

#### Train/Test Split

First, let's split the data into train and test sets by using the train_test_split function.

In [123]:
# Define the features and target variable
X = df.drop(columns='usage_rate_category')
y = df['usage_rate_category']

# Split the data so the test set contains 20% of the points, 
# (the training set will contain the rest of the points)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [124]:
# Show the shape of train and test sets
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5869755, 18) (1467439, 18) (5869755,) (1467439,)


#### Scaling Data

In [125]:
# Initialize a scaler
my_standard_scaler = StandardScaler()

# Fit the scaler on train data and then transform the train data
X_scaled_train = my_standard_scaler.fit_transform(X_train)

# Transform test data using the same scaler
X_scaled_test = my_standard_scaler.transform(X_test)

#### Handling Imbalanced Datasets

Let's take a look at our traget variables. 

In [126]:
# Get the percentage of each unique value in 'usage_rate_category' column
df['usage_rate_category'].value_counts() / len(df) * 100

usage_rate_category
2    67.176239
3    19.979613
1    12.844147
Name: count, dtype: float64

The data is imbalanced, with the majority of the usage rate category falling in class 2 (67%), followed by class 3 (20%), and class 1 (13%). 

We will apply SMOTE to the training set to address this issue since SMOTE can generate synthetic samples for the minority class.

In [127]:
# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the X_scaled_train set
X_sm_train, y_sm_train = smote.fit_resample(X_scaled_train, y_train)

In [128]:
# Print the shape of the new X_train dataset
print(X_sm_train.shape)

(11831433, 18)


In [129]:
# Print the shape of the new y_train dataset
print(y_sm_train.shape)

(11831433,)


In [130]:
# Compare the original class distribution with resampled class distribution
print('Original Class Distribution:')
display(pd.Series(y_train).value_counts().sort_index())

print('\nResampled Class Distribution:')
display(pd.Series(y_sm_train).value_counts().sort_index())

Original Class Distribution:


usage_rate_category
1     753475
2    3943811
3    1172469
Name: count, dtype: int64


Resampled Class Distribution:


usage_rate_category
1    3943811
2    3943811
3    3943811
Name: count, dtype: int64

#### Converting the Data into DataFrames

Since we have scaled and resampled our training and testing datasets, we can now proceed to convert them into training and testing DataFrames.

In [131]:
# Convert the resampled data (X_train_resampled and y_train_resampled) into a DataFrame
train_df = pd.DataFrame(np.column_stack((X_sm_train, y_sm_train)), columns=df.drop('usage_rate_category', axis=1).columns.tolist() + ['usage_rate_category'])

# Convert the test data (X_test and y_test) into a DataFrame
test_df = pd.DataFrame(np.column_stack((X_scaled_test, y_test)), columns=df.drop('usage_rate_category', axis=1).columns.tolist() + ['usage_rate_category'])

#### Train Data

In [132]:
# Look at the first 5 rows of the final train dataset
train_df.head()

,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,holiday,is_weekend_or_holiday,year,month,day,hour,minute,usage_rate_category
0,-1.375345,-0.789945,-0.667162,-0.099799,-0.428081,0.388370,0.306077,-1.406683,-0.436337,-1.014580,1.820021,-0.168116,1.482137,1.427242,-1.310568,-0.195224,1.228122,-1.303354,2.0
1,1.125367,0.647661,0.336991,0.776477,0.410568,-0.153740,0.306077,1.015281,-0.436337,0.779536,-0.548136,-0.168116,1.482137,-0.700652,0.428313,0.145410,0.216731,-0.434267,2.0
2,1.125367,1.366463,0.336991,-1.122121,0.410568,2.014700,-0.455251,-0.195701,-0.436337,-0.566051,-0.548136,-0.168116,-0.674701,1.427242,-1.600382,0.713132,-1.372598,-0.723963,1.0
3,0.083403,0.168459,-0.667162,-0.537937,1.715132,0.543259,-2.739235,-1.103938,-0.436337,0.779536,-0.548136,-0.168116,1.482137,1.427242,-1.600382,-0.535857,0.939153,-0.434267,2.0
4,0.125082,-0.310743,-0.667162,0.776477,1.062850,-0.463517,0.306077,0.712536,-0.436337,0.331007,-0.548136,-0.168116,-0.674701,-0.700652,1.007940,-0.762946,0.650184,1.593602,2.0


In [133]:
# Print the shape of the train data
train_df.shape

(11831433, 19)

In [134]:
# Sanity check to make sure we don't have any NAs
train_df.isna().sum()

station_id                        0
bikes_available                   0
dock_count                        0
mean_dew_point_f                  0
mean_humidity                     0
mean_sea_level_pressure_inches    0
mean_visibility_miles             0
mean_wind_speed_mph               0
precipitation_inches              0
cloud_cover                       0
zip_code                          0
holiday                           0
is_weekend_or_holiday             0
year                              0
month                             0
day                               0
hour                              0
minute                            0
usage_rate_category               0
dtype: int64

#### Test Data

In [135]:
# Look at the first 5 rows of the final test dataset
test_df.head()

,station_id,bikes_available,dock_count,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,precipitation_inches,cloud_cover,zip_code,holiday,is_weekend_or_holiday,year,month,day,hour,minute,usage_rate_category
0,0.750260,-1.029546,2.345298,0.922523,0.596934,0.001149,0.306077,-0.801192,2.291809,0.779536,-0.548136,-0.168116,-0.674701,-0.700652,1.007940,0.826677,-1.083629,0.724515,3.0
1,-0.416739,-0.071142,-0.667162,0.776477,1.249216,-0.540961,-0.455251,0.107045,2.291809,-0.117522,-0.703502,-0.168116,-0.674701,-0.700652,1.587567,-0.649401,-1.661567,-0.434267,2.0
2,-0.375060,-1.269147,1.341144,0.192293,-0.800813,0.853036,0.306077,0.409790,-0.436337,-0.117522,-0.091454,-0.168116,1.482137,1.427242,-1.020755,1.394399,-0.216723,-1.303354,3.0
3,1.333760,-0.550344,0.336991,0.922523,0.317385,-0.540961,0.306077,0.712536,-0.436337,0.779536,-0.548136,-0.168116,-0.674701,-0.700652,0.138499,1.735032,1.661576,0.724515,2.0
4,-1.667095,-0.550344,-0.667162,-0.537937,-0.893996,-0.463517,0.306077,0.712536,-0.436337,0.779536,1.820021,-0.168116,-0.674701,-0.700652,-0.441128,0.372499,-0.361207,0.145124,2.0


In [136]:
# Print the shape of the test data
test_df.shape

(1467439, 19)

In [137]:
# Sanity check to make sure we don't have any NAs
test_df.isna().sum()

station_id                        0
bikes_available                   0
dock_count                        0
mean_dew_point_f                  0
mean_humidity                     0
mean_sea_level_pressure_inches    0
mean_visibility_miles             0
mean_wind_speed_mph               0
precipitation_inches              0
cloud_cover                       0
zip_code                          0
holiday                           0
is_weekend_or_holiday             0
year                              0
month                             0
day                               0
hour                              0
minute                            0
usage_rate_category               0
dtype: int64

In [138]:
# Save the final train and test datasets to Parquet format
train_df.to_parquet('data/train_dataset.parquet', index=False)
test_df.to_parquet('data/test_dataset.parquet', index=False)